<a href="https://colab.research.google.com/github/camerontruong/ARA-READABILITY/blob/main/Invoice_Text_Extract_v1_09AUG21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pdfplumber

     |████████████████████████████████| 45 kB 1.5 MB/s 
     |████████████████████████████████| 5.6 MB 6.0 MB/s 
     |████████████████████████████████| 138 kB 55.4 MB/s 
     |████████████████████████████████| 1.9 MB 32.7 MB/s 
  Created wheel for pdfplumber: filename=pdfplumber-0.5.28-py3-none-any.whl size=32239 sha256=5174f24d48c59ad23ef4f8399734b3a7733fe8078e1d007ab1152d0fabf911e8
  Stored in directory: /root/.cache/pip/wheels/f2/b1/a0/c0a77b756d580f53b3806ae0e0b3ec945a8d05fca1d6e10cc1
Successfully built pdfplumber


In [3]:

import re
import requests
import pdfplumber
import pandas as pd
from collections import namedtuple

In [4]:
Inv = namedtuple('Inv', 'vend_num vend_name inv_dt due_dt inv_amt net_amt description')

In [5]:
def download_file(url):
    local_filename = url.split('/')[-1]
    
    with requests.get(url) as r:
        with open(local_filename, 'wb') as f:
            f.write(r.content)
        
    return local_filename

In [6]:
ap_url = 'https://www.tabs3.com/support/sample/apreports.pdf'

In [7]:
ap = download_file(ap_url)

In [13]:
with pdfplumber.open(ap) as pdf:
    page = pdf.pages[16]
    text = page.extract_text()
print(text)

AccountsPayableReportPack
Invoice by Vendor List
Date: 08/25/2020 Invoice by Vendor List Page: 1
Jensen, Martin & Anderson, P.C.
Inv Due Inv Disc Disc Net
Voucher # Invoice # Date Date Amount Amount Date Amount 1099 Ref # Description Codes
200 United Parcel Service
200.01MLJ 3243387 032620 062620 75.00 P 75.00 1 Overnight document delivery * 1
3243387 032720 062720 75.00 P 75.00 3 Postage * 1
3243387 072620 072620 75.00 P 75.00 2 Overnight document delivery * 1
Vendor Totals 225.00 0.00 225.00
202 Software Technology, LLC
312205 082020 082520 490.00 P 490.00 1 Software Maintenance Agreement Tabs3 * 1
312206 082020 082520 239.00 P 239.00 2 Software Maintenance Agreement Tabs3 General Ledger * 1
312207 082020 082520 239.00 P 239.00 3 Software Maintenance Agreement Tabs3 Accounts Payable * 1
312208 082020 082520 239.00 P 239.00 4 Software Maintenance Agreement Tabs3 Trust Accounting * 1
312209 082020 082520 800.00 P 800.00 5 Software Maintenance Agreement PracticeMaster (9 Users) * 1
3122

In [14]:
new_vend_re = re.compile(r'\d{3} ')


In [15]:
for line in text.split('\n'):
  if new_vend_re.match(line):
    print(line)


200 United Parcel Service
202 Software Technology, LLC
203 Clerk of the County Court
204 Clerk of the District Court
225 Jackson/Wylinda
240 NELCO
250 D & B Real Estate Management Company
300 Thomson Reuters Payment Center
325 Professional Messenger Services
400 Sprint Local & Long Distance
700 Clean All Janitorial Services


In [22]:
vend_nums =[]
vend_names = []
for line in text.split('\n'):
  if new_vend_re.match(line):
    vend_num, *vend_name = line.split()
    vend_name = ' '.join(vend_name)
    vend_nums.append(vend_num)
    vend_names.append(vend_name)
print(vend_nums)
print(vend_names)

['200', '202', '203', '204', '225', '240', '250', '300', '325', '400', '700']
['United Parcel Service', 'Software Technology, LLC', 'Clerk of the County Court', 'Clerk of the District Court', 'Jackson/Wylinda', 'NELCO', 'D & B Real Estate Management Company', 'Thomson Reuters Payment Center', 'Professional Messenger Services', 'Sprint Local & Long Distance', 'Clean All Janitorial Services']


INVOICES LINE

In [32]:
# inv_line_re = re.compile(r'(\d{6}) (\d{6}) ([\d,]+\.\d{2}) [\sP]*([\d,]+\.\d{2}) [YN ]*\d (.*?) [*\s\d]')
inv_line_re = re.compile(r'\d{6} \d{6} [\d,]+\.\d{2}')

In [39]:
for line in text.split('\n'):
  line = inv_line_re.search(line)
  if line:
    print(line.group(3))


75.00
75.00
75.00
490.00
239.00
239.00
239.00
800.00
105.00
205.00
225.00
75.00
75.00
150.00
750.00
1,500.00
129.85
7,500.00
7,500.00
10.00
4.00
75.00
40.00
4.00
50.00
50.00
50.00
50.00
20.00
25.00
47.50
35.00
68.96
750.00
750.00


In [35]:
inv_line_re = re.compile(r'\d{6} \d{6} [\d,]+\.\d{2} [\sP]')

In [ ]:
for line in text.split('\n'):
  line=  inv_line_re.search(line)
  if 

In [29]:
line_items = []
for line in text.split('\n'):
    if new_vend_re.match(line):
        vend_num, *vend_name = line.split()
        vend_name = ' '.join(vend_name)    
    
    line = inv_line_re.search(line)
    if line:
        inv_dt = line.group(1)
        due_dt = line.group(2)
        inv_amt = line.group(3)
        net_amt = line.group(4)
        desc = line.group(5)
        line_items.append(Inv(vend_num, vend_name, inv_dt, due_dt, inv_amt, net_amt, desc))

In [30]:

df = pd.DataFrame(line_items)

In [31]:
df

,vend_num,vend_name,inv_dt,due_dt,inv_amt,net_amt,description
0,200,United Parcel Service,032620,062620,75.00,75.00,Overnight document delivery
1,200,United Parcel Service,032720,062720,75.00,75.00,Postage
2,200,United Parcel Service,072620,072620,75.00,75.00,Overnight document delivery
3,202,"Software Technology, LLC",082020,082520,490.00,490.00,Software Maintenance Agreement Tabs3
4,202,"Software Technology, LLC",082020,082520,239.00,239.00,Software Maintenance Agreement Tabs3 General L...
5,202,"Software Technology, LLC",082020,082520,239.00,239.00,Software Maintenance Agreement Tabs3 Accounts ...
6,202,"Software Technology, LLC",082020,082520,239.00,239.00,Software Maintenance Agreement Tabs3 Trust Acc...
7,202,"Software Technology, LLC",082020,082520,800.00,800.00,Software Maintenance Agreement PracticeMaster ...
8,202,"Software Technology, LLC",082020,082520,105.00,105.00,Software Maintenance Agreement Tabs3 Taskbill
9,202,"Software Technology, LLC",082520,083020,205.00,205.00,Purchase Tabs3 Device Interface Program
